In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
#Getting rid of cases that have no labels.Part of data cleaning essentially. Could try to do something fancy with missing
#label but they are only 20 cases from like 20k so no point in spending time on that IMO.
train_skiprows = [
    94,
    253,
    510,
    1528,
    1940,
    1954,
    4030,
    4427,
    4644,
    4728,
    5232,
    5763,
    6296,
    6334,
    6704,
    7084,
    9478,
    9676,
    10000,
    10489,
    10737,
    10911,
    11675,
    12281,
    12600,
    13148,
    14168,
    14723,
]


In [3]:
#Read data. Csv so seperate with ','.Skip the rows that dont have columns.
raw_data = pd.read_csv('../data/raw/train.csv',sep = ',',skiprows=train_skiprows)

In [4]:
#Put labels into a list (one big string atm) with split and covert them to integers.
raw_data["labels"] = (raw_data["labels"].str.split(",").apply(lambda x: list(map(int, x))))

In [5]:
raw_data.head()

,ex_id,labels,features
0,0,"[446, 521, 1149, 1249, 1265, 1482]",0:0.084556 1:0.138594 2:0.094304 3:0.195764 4:...
1,1,"[78, 80, 85, 86]",0:0.050734 1:0.762265 2:0.754431 3:0.065255 4:...
2,2,"[457, 577, 579, 640, 939, 1158]",0:0.101468 1:0.138594 2:0.377215 3:0.130509 4:...
3,3,"[172, 654, 693, 1704]",0:0.186024 1:0.346484 2:0.141456 3:0.195764 4:...
4,4,"[403, 508, 1017, 1052, 1731, 3183]",0:0.135290 1:0.277187 2:0.141456 3:0.065255 4:...


In [6]:
#Convert features (a string atm) into a dictionary where key is a integer and the value is a float.
#First split converts each row into a list, then within each list we loop the elements and seperate by : for which the first 
#value is the key and the second is the value.
raw_data["features"] = (raw_data["features"].str.split(" ").apply(lambda x: {int(i.split(":")[0])
                                                                             : float(i.split(":")[1]) for i in x}))

In [7]:
raw_data.head()

,ex_id,labels,features
0,0,"[446, 521, 1149, 1249, 1265, 1482]","{0: 0.084556, 1: 0.138594, 2: 0.094304, 3: 0.1..."
1,1,"[78, 80, 85, 86]","{0: 0.050734, 1: 0.762265, 2: 0.754431, 3: 0.0..."
2,2,"[457, 577, 579, 640, 939, 1158]","{0: 0.101468, 1: 0.138594, 2: 0.377215, 3: 0.1..."
3,3,"[172, 654, 693, 1704]","{0: 0.186024, 1: 0.346484, 2: 0.141456, 3: 0.1..."
4,4,"[403, 508, 1017, 1052, 1731, 3183]","{0: 0.13529, 1: 0.277187, 2: 0.141456, 3: 0.06..."


In [8]:
#Converts the dictionary corresponding to each row into a list that has the feature keys in one list and the values in another
raw_data["features"] = raw_data["features"].apply(lambda x: [list(x.keys()), list(x.values())])

In [9]:
NUM_FEATURES = 5000
NUM_CLASSES = 3993

#Create the two sparse matrices, one for the features and one for the classes.
temp = np.zeros((raw_data.shape[0], NUM_FEATURES))
features = pd.DataFrame(temp)

temp = np.zeros((raw_data.shape[0], NUM_CLASSES))
labels = pd.DataFrame(temp)

In [10]:
raw_data.head()

,ex_id,labels,features
0,0,"[446, 521, 1149, 1249, 1265, 1482]","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 19, 2..."
1,1,"[78, 80, 85, 86]","[[0, 1, 2, 3, 4, 6, 10, 11, 13, 15, 16, 18, 21..."
2,2,"[457, 577, 579, 640, 939, 1158]","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."
3,3,"[172, 654, 693, 1704]","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."
4,4,"[403, 508, 1017, 1052, 1731, 3183]","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14..."


In [11]:
#Puts the weight of each feature per example into the empty sparse matrix.
def create_features(row):
    features.loc[row.name, row["features"][0]] = row["features"][1]

#Puts a 1 on each row of the matrix that has label at that column.
def create_labels(row):
    labels.loc[row.name, row["labels"]] = 1

In [12]:
raw_data.apply(lambda x: create_features(x), axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
15506    None
15507    None
15508    None
15509    None
15510    None
Length: 15511, dtype: object

In [ ]:
raw_data.apply(lambda x: create_labels(x), axis=1)

In [ ]:
Path("../data/expanded").mkdir(parents=True, exist_ok=True)

In [ ]:
file_prefix = "train"
features.to_csv( f"../data/expanded/{file_prefix}_features.csv", index=False, header=False)
labels.to_csv(f"../data/expanded/{file_prefix}_labels.csv", index=False, header=False)
print(f"Saved dataframes as {file_prefix}_features.csv and {file_prefix}_labels.csv")